In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# matplotlib 한글 패치
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
    rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin': # Mac
    rc('font', family='AppleGothic')
else: #linux
    rc('font', family='NanumGothic')

## 파일 불러오기

In [4]:
# .. 상위 폴더
train_pd = pd.read_csv('../../../../data/train.csv')
train_pd['simple_label'] = train_pd['label'].apply(lambda x: int(x))
train_pd

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,2
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,4
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,2
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,3
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,0
...,...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0,0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,4.2,1.0,4
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,0.6,0.0,0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0,3


## 대회 설명

![label_description](../../../src/label_exp.png)

## 정성적 분석, simple label

In [5]:
train_pd.where(train_pd['simple_label']==0).dropna().head(15)

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,0.0
7,boostcamp-sts-v1-train-007,nsmc-sampled,이렇게 귀여운 쥐들은 처음이네요.ㅎㅎㅎ,이렇게 지겨운 공포영화는 처음..,0.6,0.0,0.0
8,boostcamp-sts-v1-train-008,petition-sampled,미세먼지 해결이 가장 시급한 문제입니다!,가장 시급한 것이 신생아실 관리입니다!!!,0.4,0.0,0.0
14,boostcamp-sts-v1-train-014,slack-sampled,아래는 게임 내에서 함께 팀을 맺고 캡쳐한 이미지입니다.,여러분들간의 다양한 아이디어를 제안하고 나누는 채널입니다.,0.0,0.0,0.0
15,boostcamp-sts-v1-train-015,petition-sampled,LG전자 스마트폰 생산 중단 청원,LG전자서비스의 노조 와해를 조사해주세요.,0.4,0.0,0.0
19,boostcamp-sts-v1-train-019,petition-sampled,내년 러시아월드컵 자진포기 청원드립니다.,2023 아시안컵 유치에 공동개최를 반대 합니다.,0.0,0.0,0.0
20,boostcamp-sts-v1-train-020,slack-sampled,앞머리 새로 하셨습니다. ^^,가방에 넣어 다니면서 조금씩 먹습니다. ^^,0.0,0.0,0.0
22,boostcamp-sts-v1-train-022,slack-sampled,답답할 때 보면 속이 뻥 뚫릴 것 같아요,양보단 한입 먹는 순간 고삐 풀릴 것 같아요 ㅋㅋㅋ,0.0,0.0,0.0
23,boostcamp-sts-v1-train-023,nsmc-sampled,노래와 잘 어우러지는 영상 덕분인지 짧지만 강한 인상이 남네요..,조금유치하지만 가볍게 볼수는 있는영화네요!,0.0,0.0,0.0
26,boostcamp-sts-v1-train-026,petition-sampled,전두환을 처벌해주세요,이재용을 구속해주세요,0.2,0.0,0.0


### 라벨 0
##### 두 문장의 핵심 내용이 동등하지 않고, 부가적인 내용에서도 공통점이 없음
- 주요 특징
    - 문장 구조 유사 (ex. 나이가 문제인가요? - 저는 무엇인가요?)
    - 비문법적 표현 동일 (ex. ^^, -_-;)
    - 특정 단어가 동일 (ex. 이렇게, 시급한, 엑소엘, 해주세요.)
    - 특정 품사의 단어가 동일해 보이나 예외가 존재 (ex. 뿌듯뿌듯 - 뿌뿌뿌)
    - 동사의 경우, ~하다, ~이다 외의 중심적인 의미를 가지는 동사는 두 문장이 상이  
    #
- 핵심 내용
    - 두 문장의 내용이 완전히 상이함   
    #
- 부가적인 내용
    - 부가적인 내용이 없기 때문에, 현재 차이를 확인하기 어려움

In [6]:
train_pd.where(train_pd['simple_label']==1).dropna().head(15)

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
13,boostcamp-sts-v1-train-013,nsmc-sampled,글쎄.. 특색도 없고 별로였던 걸로 기억하는데..,큰 반전 없던걸로 기억하는데...,1.8,0.0,1.0
35,boostcamp-sts-v1-train-035,slack-sampled,또 먹어 보고 싶네요.,얼마나 드시고 싶었으면…,1.2,0.0,1.0
48,boostcamp-sts-v1-train-048,petition-sampled,알콜중독자가 기초생활수급자?,기초생활수급자 장애인을 위하여.,1.0,0.0,1.0
53,boostcamp-sts-v1-train-053,nsmc-sampled,"다만, 관객에 대한 배려가 없다.",이 정도 수준으로 영화관에 내보내는건 관객에 대한 모독이다.,1.8,0.0,1.0
56,boostcamp-sts-v1-train-056,nsmc-sampled,제일 좋은것같던데..,난이게 제일 재밌었는데..,1.6,0.0,1.0
60,boostcamp-sts-v1-train-060,slack-sampled,책도 앞부분을 읽어보니 우리 회사 문화와 업무 스타일에 좋은 팁들이 들어있더라구요!!,첫날 부터 우리 회사의 좋은 문화를 가득 체험하고 갑니다 ㅎㅎ,1.0,0.0,1.0
61,boostcamp-sts-v1-train-061,petition-sampled,그루밍 성범죄'법이 있어야 합니다.,"!!'그루밍 범죄', 청소년 성폭력 범죄에서 절반!!",1.0,0.0,1.0
73,boostcamp-sts-v1-train-073,nsmc-sampled,감성을 자극하진 못한다.,감성을 자극했던 좋은영화,1.6,0.0,1.0
75,boostcamp-sts-v1-train-075,petition-sampled,문재인님불만하지마세요,문재인님제발욕설하지마세요,1.6,0.0,1.0
89,boostcamp-sts-v1-train-089,slack-sampled,여기 왕갈비 드시고 이 광경을 구경하시면 정말 멋지죠.,왕갈비 최고였습니다. 정자에 부는 시원한 바람 또한 일품이었습니다. 최고최고~~,1.4,0.0,1.0


### 라벨 1
##### 두 문장의 핵심 내용은 동등하지 않지만, 비슷한 주제를 다루고 있음
- 주요 특징
    - 특정 체언(명사, 대명사, 수사)이나 수식언(관형사, 부사, 관사, 접속사)가 동일
    - 주어나 목적어가 같은 경우가 많지만 예외가 존재
    - 용언(동사, 조동사, 형용사, 계사)이 상이, 부정 부사 및 부정 용언이 다른 경우도 있음
    - 의미적 방향성이 다름, 유사한 주제에 대해 서로 다른 말을 하고 있음
    - 예시에 한해서 라벨 0 데이터에 비해 라벨 1 데이터의 구조적 차이가 큰 것으로 보임
    #
- 핵심 내용
    - 앞서 언급한 바와 같이 유사 주제에 대한 다른 이야기, 방향성이 다름
    #
- 부가적인 내용
    - 1에서 부가적인 내용이 포함한다는 구체적인 내용이 명시되지 않았음
    - 핵심 내용 : 용언, 부가적 내용 : 수식언 과 같이 부가적인 내용은 보조적인 정보에 대한 것이지 않을까 싶음
    #
- 비고
    - 구조적 유사성(SVO와 같은 문장 구성 성분의 순서)가 중요하지 않은 것 같음
    - 의미적 유사성 또는 동일성이 주요 관심사

In [7]:
train_pd.where(train_pd['simple_label']==2).dropna().head(15)

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,2.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,2.0
5,boostcamp-sts-v1-train-005,nsmc-rtt,오마이가뜨지져스크롸이스트휏,오 마이 갓 지저스 스크론 이스트 팬,2.6,1.0,2.0
10,boostcamp-sts-v1-train-010,slack-rtt,그 책부터 언능 꺼내봐야 겠어요!,책에서 꺼내야겠어요!,2.4,0.0,2.0
39,boostcamp-sts-v1-train-039,petition-sampled,휴교령을 내려주세요,대구광역시 휴교령 내려주십시요,2.8,1.0,2.0
63,boostcamp-sts-v1-train-063,slack-sampled,부트캠프 프로그램 얘기부터 추천 도메인 얘기까지 순식간에 시간이 흘러가서 너무 아쉬...,각자 업무와 부트캠프 얘기부터 앞으로 진로 얘기까지 나누다보니 한시간 훌쩍 넘게 수...,2.0,0.0,2.0
66,boostcamp-sts-v1-train-066,slack-sampled,식당도 진짜 멋졌어요,정말 너무 멋집니다!!,2.8,1.0,2.0
78,boostcamp-sts-v1-train-078,slack-rtt,과연 나중에 다시 작업을 통해 만나 뵙게 될 수 있을지 궁금해하며 마무리했습니다.,앞으로 일을 통해 다시 만날 수 있을까 하는 생각이 들었다.,2.4,0.0,2.0
80,boostcamp-sts-v1-train-080,petition-sampled,가상화페규제 반대합니다,가상화페 거래소 패지 반대,2.8,1.0,2.0
92,boostcamp-sts-v1-train-092,nsmc-sampled,그냥 재밌으니까 ㅋ,재밌으니까 좀 봐요들 ㅋㅋ,2.8,1.0,2.0


### 라벨 2
##### 두 문장의 핵심 내용은 동등하지 않지만, 몇 가지 부가적인 내용을 공유함
- 주요 특징
    - 특정 체언이 동일하나 주어나 발화 대상을 공유하지 않음
    - 수식언에 의해 특정되는 주체가 다름, 주체의 범위가 다름
    - 용언이 다른 경우도 존재함
    #
- 핵심 내용
    - 문장의 주체가 되는 핵심 대상이 다르거나 불명확함
    #
- 부가적인 내용
    - 부가적인 정보, 주어 또는 명사의 존재나 용언으로 대표되는 행위의 유사성
    #
- 비고
    - 문장을 여러 개의 하부 문장으로 쪼개거나 문장 내의 정보를 나눠서 비교하는 게 나아보임  
    ***
    - 라벨을 나누는 기준이 5개 존재하고 그 기준을 각기 어느 정도의 유사성으로 통과하느냐에 따라 score가 나오는 것이 아닐까
    - QA 방식의 접근
    ex. 문장의 주어가 무엇인가?
        - id 0의 경우, 문장 1과 문장 2의 주어 불명확
        - id 127의 경우, 문장 1 (여러분)과 문장 2(권한이 있는 특정 인물)이 다름
        - 라벨 2는 주어가 불명확하거나 다른 것 같음
    ex. 문장의 목적어가 무엇인가?
        - id 10의 경우, 문장 1 (책)과 문장 2 (불명확)과 다름
        - id 39의 경우, 문장 1 (휴교령)과 문장 2 (대구 광역시 휴교령)이 유사함  
    ***
    - 각각의 질의에 대해 나온 답에 대한 correlation 등의 score를 구해 합친 것이 아닐까 생각됨 (각각 최대 1)
    - 질의의 유형은 품사에 대한 것 또는 문장 성분에 대한 것일 듯함
        - 품사 : 체언, 용언, 수식언, 관계언, 독립언
        - 문장 성분 : 주어, 동사, 목적어, 보어, 수식어
        - 문장 성분이 유력해 보이나 그것이 아니더라도 라벨을 나누는 명확한 기준 또는 질문이 존재할 것이라 생각됨
        - 물론 가설임

In [8]:
train_pd.where(train_pd['simple_label']==3).dropna().head(15)

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,3.0
6,boostcamp-sts-v1-train-006,slack-rtt,전 암만 찍어도 까만 하늘.. ㅠㅠ,암만 찍어도 하늘은 까맣다.. ㅠㅠ,3.6,1.0,3.0
11,boostcamp-sts-v1-train-011,nsmc-sampled,극의전개가너무느릿하여답답합니다,전개가 너무 느려서 속터진다,3.8,1.0,3.0
12,boostcamp-sts-v1-train-012,nsmc-rtt,오랫만에 존쿠삭의 연기보니 좋네요 Good~~!!,오랜만에 존 쿠삭의 연기를 보니 반갑다. 좋아요~~!!,3.8,1.0,3.0
16,boostcamp-sts-v1-train-016,slack-sampled,환갑이면? ㅎㅎㅎ 이런 재미난 얘기하느라 시간이 금방 가더라고요.,넘넘 즐거운 이야기를 하고나니 시간이 금방 가버렸네요.,3.0,1.0,3.0
17,boostcamp-sts-v1-train-017,petition-rtt,최종구 금융위원장을 파면하라!,최종구 금융위원장 해고!,3.8,1.0,3.0
28,boostcamp-sts-v1-train-028,slack-rtt,겨울산이 예쁘지만 산을 잘 못타서 대리만족 중입니당,겨울산은 예쁜데 제가 등산을 잘 못해서 대만족입니다.,3.2,1.0,3.0
29,boostcamp-sts-v1-train-029,nsmc-rtt,한사람의 파멸을 적나라하게 드러내 준 영화,한 사람의 파멸을 드러내는 영화,3.6,1.0,3.0
33,boostcamp-sts-v1-train-033,nsmc-sampled,잘 만든 영화군요.,잘 만든 영화라고 생각한다.,3.6,1.0,3.0
36,boostcamp-sts-v1-train-036,petition-rtt,보험이 재산압류를 해서는 안 될 일입니다.,보험은 재산을 몰수해서는 안됩니다.,3.4,1.0,3.0


### 라벨 3
##### 두 문장의 핵심 내용은 대략적으로 동등하지만, 부가적인 내용에 무시하기 어려운 차이가 있음
- 주요 특징
    - 부분 문장의 체언 혹은 주어, 목적어가 동일, 같은 대상에 대한 문장
    - 부분 문장의 용언 혹은 동사가 동일하거나 유사함, 순서가 다르거나 표현이 다름
    #
- 핵심 내용
    - 유사한 내용으로 보이지만 그 표현이 미묘하게 다름
    - 체언에 대한 수식이 다를 수도 있고 용언들이 유의어 관계일 수도 있음
    #
- 부가적인 내용
    - 무시하기 어려운 차이에 집중
    - 수식 표현이 다른 것이나 용언 표현이 다른 것이지 않을까 추정
    - id 3의 경우, 입사 후 - 화상으로만 보다가 리얼로, 상황 자체가 다르다.
    - id 64의 경우, 그렉으로 전시한 사진 - 그렉과 함께 전시한 사진으로 수식에 의해 그 대상 정의가 달라진다.
        -근데 이거 차이를 인지했나? 조금 의아하다.
    - 품사나 문장 성분 외에도 문장을 파악하는 방식이 있는지 찾아볼 필요가 있다.
    #
- 비고
    - label이 5개의 질의에 대한 답의 유사도 합이라는 가설 하에 각 질의에 대한 답이 문장 1, 2 모두에 없을 경우 어떻게 되는 것인가
    - 핵심 내용에 대한 score를 높게 주는 것인지 총 score인 5에 맞춰서 scaling 하는 것인지 불명확
    - id 59의 이건 진짜 대박임 ㅇㅇ 과 이건 진짜 최고임을 비교했을 때, 이 둘은 이거 / 진짜 / 대박-최고 / 이다. 로 보어 표현이 다르고 목적어가 존재하지 않는데, 보어 표현이 단어 임베딩에서 갖는 거리(?)나 유사도가 0.8인 건가 싶다.
    - 사실 짧은 문장에서 score 가중이나 scaling을 고민해보려 했는데 뭔가 맞아버린 느낌...
    - id 52의 경우와 같이 주된 문장과 보조적인 문장이 함께 있는 겹문장의 경우 판별이 어떻게 되는 것인가 궁금하다.

In [9]:
train_pd.where(train_pd['simple_label']==4).dropna().head(15)

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,4.0
9,boostcamp-sts-v1-train-009,petition-sampled,크림하우스 환불조치해주세요.,크림하우스 환불조치할 수 있도록해주세여,4.2,1.0,4.0
18,boostcamp-sts-v1-train-018,petition-rtt,정말 간절히 정부의 도움이 필요합니다.,정부의 도움이 절실히 필요합니다.,4.0,1.0,4.0
21,boostcamp-sts-v1-train-021,petition-rtt,김기덕 조재현 성폭행 철저히 수사해주세요!,김기덕·조재현 성폭행 의혹 철저히 수사하라!,4.2,1.0,4.0
24,boostcamp-sts-v1-train-024,nsmc-rtt,군대가기전에 봤었는데 진짜 윈터스같은 사람이 상관이면 목숨걸고 싸워도 후회는 없지 ...,입대하기 전에 봤는데 윈터스 같은 사람이 진심으로 아껴준다면 목숨을 걸고 싸워도 후...,4.2,1.0,4.0
25,boostcamp-sts-v1-train-025,petition-rtt,국민청원에 올린글 삭제하는 청와대뉴미디어정책실은 억울한 피해자를 죽이고 경찰에게 증...,"국민청원 글을 삭제하는 청와대 뉴미디어정책실은 부당한 피해자를 살해하고, 경찰에 증...",4.0,1.0,4.0
27,boostcamp-sts-v1-train-027,slack-rtt,"마지막으로 리모트 근무의 장점에 대해 이야기했는데, 시간을 효율적으로 사용할 수 있...","마지막으로 재택근무의 장점에 대해 이야기를 나누었고, 시간을 효율적으로 사용할 수 ...",4.2,1.0,4.0
30,boostcamp-sts-v1-train-030,petition-rtt,금융워원회의 공모주 개인 배정 축소(폐지)를 막아주세요,금융감독원의 공모주 개인배분 축소(폐지)를 막아주세요.,4.4,1.0,4.0
31,boostcamp-sts-v1-train-031,slack-rtt,지금 아주 만족해요.,지금은 매우 만족합니다.,4.4,1.0,4.0
32,boostcamp-sts-v1-train-032,slack-sampled,저도 너무 좋아하는 노래ㅎㅎㅎ,오! 저도 좋아하는 노래 ㅎㅎ,4.0,1.0,4.0


### 라벨 4
##### 두 문장의 핵심 내용이 동등하며, 부가적인 내용에서는 미미한 차이가 있음
- 주요 특징
    - 부사나 감탄사가 다르다.
    #
- 비고
    - 앞 부분의 문장 구성 성분이 틀렸었다. 인터넷 블로그 글을 믿는 게 아니었다.
    - 문장 주성분과 부속 성분으로 나뉘는 게 맞는 것 같다.
    - 링크 : [한국어의 문장 성분](https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B5%AD%EC%96%B4%EC%9D%98_%EB%AC%B8%EC%9E%A5_%EC%84%B1%EB%B6%84)
    - 주성분이 4개의 QA이고 부속 성분과 독립 성분이 1개의 QA를 차지하지 않을까 싶다.
    - 단순 머신 러닝으로 해결할 수 있나! 싶었지만 semantic relation을 가지는 단어들을 비교하기 위해서라도 딥러닝이 필요해 보이기는 한다. (embedding 때문)
    - 그래도 이 발견은 유의미하다.

In [10]:
train_pd.where(train_pd['simple_label']==5).dropna().head(15)

,id,source,sentence_1,sentence_2,label,binary-label,simple_label
342,boostcamp-sts-v1-train-342,petition-rtt,정형식판사 감사요청,정형식 판사 감사요청,5.0,1.0,5.0
358,boostcamp-sts-v1-train-358,petition-sampled,이병박 출국금지 청원 합니다.,이명박 출국 금지 청원합니다.,5.0,1.0,5.0
448,boostcamp-sts-v1-train-448,petition-rtt,제주도 가정용lpg통가스보일러,제주도 가정용 LPG 통가스보일러,5.0,1.0,5.0
621,boostcamp-sts-v1-train-621,petition-sampled,청소년법 폐지 시켜주세요,청소년법 폐지 시켜 주세요,5.0,1.0,5.0
624,boostcamp-sts-v1-train-624,nsmc-rtt,"잔잔한 감동으로 다가온영화,","잔잔한 감동으로 다가온 영화,",5.0,1.0,5.0
760,boostcamp-sts-v1-train-760,nsmc-rtt,그냥재미없음;;;;,그냥 재미 없음;,5.0,1.0,5.0
846,boostcamp-sts-v1-train-846,petition-rtt,미친 부동산(서울 아파트),미친 부동산 (서울 아파트),5.0,1.0,5.0
996,boostcamp-sts-v1-train-996,petition-rtt,이명박 출국금지 구속수사!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,이명박 출국금지 구속 수사!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,5.0,1.0,5.0
1015,boostcamp-sts-v1-train-1015,petition-rtt,카쉐어링 업체의 무료편도를 제한해주세요,카셰어링 업체의 무료 편도를 제한해주세요,5.0,1.0,5.0
1054,boostcamp-sts-v1-train-1054,petition-rtt,이수역 폭행 피해자 코스프레 제대로수사,이수역 폭행 피해자 코스프레 제대로 수사,5.0,1.0,5.0


### 라벨 5
##### 두 문장의 핵심 내용이 동일하며, 부가적인 내용들도 동일함
- 주요 특징
    - 띄어씌기와 문장 부호(~,!)에서만 차이를 보임
    #
- 비고
    - 띄어쓰기가 라벨에 영향을 끼치지 못하므로 이에 대한 전처리가 불필요해보인다.
    - base 코드의 tokenization 출력문을 확인할 필요가 있다.
    - 추후 작업을 통해 label의 의미를 명확하게 파악할 수 있다면 그 의미 값들만 토큰 형태로 최종 모델에 전달하는 것도 고려할 수 있겠다.
    - 전처리가 들어간다면 그 부분에 들어갈 것이고 NLU에 해당하는 과제라고 볼 수 있겠다.

## 정리

### 라벨의 의미
- (가설) 문장의 주성분 / 문장의 부속성분 + 문장의 독립성분을 토대로 한 주어진 두 문장의 correlation score의 합
- 최종 score가 corrleation score인 것도 연관이 있지 않을까?
- 링크 : [한국어의 문장 성분](https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B5%AD%EC%96%B4%EC%9D%98_%EB%AC%B8%EC%9E%A5_%EC%84%B1%EB%B6%84)

### further work
- label 가설의 정립
    - base 코드 tokenization 출력 확인
    - 한국어 pos tagging 포함 tokenization
    - token embedding 시각화
    - label의 의미 명확화
- NLU
    - NLU 모델 설계(feature extraction)
    - 전처리, 데이터 증강
    - 과정 별 모델 성능 비교